# Week 2: RAG

Two methods to equip a model with new knowledge:
1. **RAG (Retrieval-Augmented Generation)**
2. Fine-tuning

---

In this tutorial, we will:  
1. Demonstrate the limitations of LLMs with examples.
2. Build a RAG using LangChain and LangGraph.
3. Enable the LLM's web search functionality.  

## Table of Contents

**建議使用 Colab 側邊欄的 TOC 瀏覽章節**

- [RAG workflow](#rag-workflow)
- [1. 安裝與啟動 Ollama (Install and Start Ollama)](#1-安裝與啟動-ollama-install-and-start-ollama)
- [2. 使用 requests 發送 prompt (Use requests to send prompt)](#2-使用-requests-發送-prompt-use-requests-to-send-prompt)
  - [2.1 The LLM Model we're going to use](#21-the-llm-model-were-going-to-use)
- [3. Limitations of Language Models](#3-limitations-of-language-models)
  - [3.1 Hallucinations](#31-hallucinations)
    - [3.1.1 Let's try in legal fields](#311-lets-try-in-legal-fields)
  - [3.2 Knowledge cut-off](#32-knowledge-cut-off)
    - [3.2.1 LLMs won't know about things that happened recently](#321-llms-wont-know-about-things-that-happened-recently)
- [4. Let's build RAG to solve it!](#4-lets-build-rag-to-solve-it)
  - [4.1 The Knowledge we want LLM to know](#41-the-knowledge-we-want-llm-to-know)
  - [4.2 Web Scraping these Knowledge](#42-web-scraping-these-knowledge)
  - [4.3 Splitting and Chunking Data](#43-splitting-and-chunking-data)
  - [4.4 Setup Embedding Model](#44-setup-embedding-model)
  - [4.5 Vector Database](#45-vector-database)
  - [4.6 Save embedding into Vector DB](#46-save-embedding-into-vector-db)
  - [4.7 Define Prompt](#47-define-prompt)
  - [4.8 Building RAG Workflow](#48-building-rag-workflow)
  - [4.9 Done! Let's try it!](#49-done-lets-try-it)
    - [4.9.1 Some question you can try](#491-some-question-you-can-try)
    - [4.9.2 Trying harder question](#492-trying-harder-question)
- [5. RAG Use Case Overview](#5-rag-use-case-overview)
- [6. Enable Web Search Functionalities](#6-enable-web-search-functionalities)


## RAG workflow

![RAG workflow](./Images/1_8_4t5Rno_9lQaMpmR33g7g.webp)  






# 1. 安裝與啟動 Ollama (Install and Start Ollama)

請先安裝 [Ollama](https://ollama.com) 並拉取模型 [`gemma3:27b`](https://ollama.com/library/gemma3:27b)。  
(Please install Ollama and pull the model `gemma3:27b`.)


# 2. 使用 requests 發送 prompt (Use requests to send prompt)


## 2.1 The LLM Model we're going to use

In [8]:
import os
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(
    model="gemma3",
    temperature=1.0,
    max_tokens=None,
    base_url="http://localhost:11434",
    timeout=None,
    max_retries=2,
    # other params...
)


In [5]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
# 準備訊息列表
messages = [
    SystemMessage(content="You are a cat and can only communicate by meowing."),
    HumanMessage(content="hi!"),
]

# 呼叫模型
response = llm.invoke(messages).content
print(response)

Meow! 🐾



# 3.Limitations of Language Models

## 3.1 Hallucinations

Language models sometimes generate responses that appear plausible but are factually incorrect or entirely fabricated. This phenomenon, known as "hallucination," can mislead users, especially in contexts requiring high accuracy, such as medical, legal, or technical fields.

### 3.1.1 Let's try in legel fields

Truth is, **there is no Article 1226 in Civil Code**, we can verify it.

[Civil Code Search(Chinese ver)](https://law.moj.gov.tw/LawClass/LawSearchCNKey.aspx?BTNType=NO&pcode=B0000001)

[Civil Code Search(English ver)](https://law.moj.gov.tw/ENG/LawClass/LawSearchCNKey.aspx?BTNType=NO&pcode=B0000001)

In [6]:
messages = [
    ("human", "What is the specific content of Article 1226 in the Civil Code(民法) of Taiwan? Reply in Chinese."),
]
ai_msg = llm.invoke(messages)

print(ai_msg.content)

Okay, let's break down the content of Article 1226 of the Civil Code (民法) of Taiwan.

**中華民國民法 第1226條**

**要約契約**

(一) 契約經當事方約定，於一定時間內，任何一方得在一定條件下解除契約。
(二)  此種契約之內容、條件及解除方式，由當事方約定。

**Translation & Explanation:**

**Article 1226: Option Contract (要約契約)**

(1) By agreement of the parties, a contract may be terminated by any party under certain conditions within a specified period.
(2) The content, conditions, and manner of termination of this type of contract shall be agreed upon by the parties.

**Key Points & Breakdown:**

* **What it is:** Article 1226 establishes the concept of an “option contract” (要約契約). This is a contract where one party (the option holder) has the *right*, but not the obligation, to enter into a main contract under specific conditions and within a specified timeframe.

* **Right to Terminate:** The core of this article is the power granted to *either* party to terminate the contract.  This isn't a standard, automatically-terminating contract.

* **Specified Peri

## 3.2 Knowledge cut-off

The knowledge of a language model is limited to the data it was trained on, up to a specific cut-off date. As a result, it cannot provide information about events, discoveries, or updates that occurred after that point, making it less reliable for addressing recent developments.

### 3.2.1 LLMs won't know about things that happened recently

The 47th United States presidential election took place on **November 5, 2024**.

The model we're using, `gemini-2.0-flash`, knowledge cutoff at **June 2024** ([see more about model](https://deepmind.google/technologies/gemini/flash/))

In [7]:
messages = [
    ("human", "Who won the 47th US President election?"),
]
ai_msg = llm.invoke(messages)

print(ai_msg.content)

Joe Biden won the 47th US Presidential election (the 2020 election). 

He defeated Donald Trump in a very close race.


# 4.Let's build RAG to slove it!

## 4.1 The Knowledge we want LLM to know

These are news reports about events that occurred between late 2024 and early 2025, which the LLM is not yet aware have taken place.

1. [Trump seeks to force TSMC negotiations, experts say](https://www.taipeitimes.com/News/biz/archives/2025/02/10/2003831601) - 專家：川普試圖迫使台積電進行談判
2. [Instagram ‘Teen Accounts’ go live in Taiwan today](https://www.taipeitimes.com/News/taiwan/archives/2025/02/11/2003831708) - Instagram「青少年帳號」今日在台灣上線
3. [Donald Trump wins US presidency](https://www.taipeitimes.com/News/front/archives/2024/11/07/2003826511) - 川普當選美國總統
4. [Team Taiwan claim U-12 Asian baseball title](https://www.taipeitimes.com/News/front/archives/2024/11/30/2003827724) - 台灣隊奪得U-12亞洲棒球錦標賽冠軍
5. [What is DeepSeek and why is it disrupting the AI sector?](https://www.taipeitimes.com/News/lang/archives/2025/02/11/2003831649) - 什麼是DeepSeek？為何它正在顛覆AI產業？

## 4.2 Web Scraping these Knowledge

Use web scraping on the five news articles mentioned above to extract the textual content of the reports.

In [9]:
import bs4
os.environ['USER_AGENT'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
from langchain_community.document_loaders import WebBaseLoader


# Load and chunk contents
loader = WebBaseLoader(
    web_paths=(
        "https://www.taipeitimes.com/News/biz/archives/2025/02/10/2003831601",
        "https://www.taipeitimes.com/News/taiwan/archives/2025/02/11/2003831708",
        "https://www.taipeitimes.com/News/front/archives/2024/11/07/2003826511",
        "https://www.taipeitimes.com/News/front/archives/2024/11/30/2003827724",
        "https://www.taipeitimes.com/News/lang/archives/2025/02/11/2003831649"
        ),
    )

docs = loader.load()

assert len(docs) == 5

print("Finished crawing news form urls.")

for i, doc in enumerate(docs):
    print(f"News {i + 1}:")
    print(f"  Source URL: {doc.metadata['source']}")
    print(f"  Total Characters: {len(doc.page_content.strip())}")
    print("-" * 30)

Finished crawing news form urls.
News 1:
  Source URL: https://www.taipeitimes.com/News/biz/archives/2025/02/10/2003831601
  Total Characters: 8696
------------------------------
News 2:
  Source URL: https://www.taipeitimes.com/News/taiwan/archives/2025/02/11/2003831708
  Total Characters: 6596
------------------------------
News 3:
  Source URL: https://www.taipeitimes.com/News/front/archives/2024/11/07/2003826511
  Total Characters: 8827
------------------------------
News 4:
  Source URL: https://www.taipeitimes.com/News/front/archives/2024/11/30/2003827724
  Total Characters: 7519
------------------------------
News 5:
  Source URL: https://www.taipeitimes.com/News/lang/archives/2025/02/11/2003831649
  Total Characters: 10852
------------------------------


## 4.3 Splitting and Chunking Data

There is too much characters here, and providing all of it to the LLM at once is not a good choice. Therefore, we need to break those into smaller chunks while preserving the semantics.

- `chunk_size`: The maximum size of each chunk.
- `chunk_overlap`: The number of overlapping characters between consecutive chunks. This ensures semantic continuity across chunks and prevents loss of context.

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split all news into {len(all_splits)} sub-documents.")

Split all news into 68 sub-documents.


## 4.4 Setup Embedding Model

The Embedding Model can convert the semantics of a sentence into a high-dimensional vector.

In [24]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")  # 或其他支援 embedding 的模型
print(embeddings.embed_query("What's our Q1 revenue?"))


[0.018574804067611694, 0.8912683129310608, -4.892469882965088, 0.06046918034553528, 0.5157424807548523, 0.008302665315568447, 1.1618986129760742, -0.49466341733932495, -0.02313239872455597, -0.45999401807785034, 1.323664903640747, 0.9787083864212036, 1.0707542896270752, -0.6464206576347351, 0.6906968951225281, 0.2472461313009262, -0.4303080439567566, -0.9496088624000549, 0.1534976214170456, 0.31445491313934326, -0.7074934244155884, -0.9292242527008057, -1.0128780603408813, -0.8978955149650574, 1.8204210996627808, 1.0105760097503662, -0.40470147132873535, 0.24186968803405762, -0.012563611380755901, -0.17961257696151733, -0.2648887634277344, 0.07651873677968979, -0.08272318542003632, -0.8771931529045105, -0.9737500548362732, 1.2908921241760254, -0.005311517044901848, -0.2892124354839325, 0.10798914730548859, -1.9025256633758545, -0.193605437874794, -1.407510757446289, -0.4320368766784668, 0.31814074516296387, 1.0021618604660034, 0.5543139576911926, 0.3231140077114105, 0.4688602387905121,

## 4.5 Vector Database

In this tutorial, for the sake of convenience and speed, we will directly use RAM as the storage location for vector data.

In [25]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

## 4.6 Save embedding into Vector DB

In [26]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

## 4.7 Define Prompt

This prompt is used in the generation stage of RAG, providing instructions to the LLM to generate a coherent and fluent response.

In [27]:
# Define prompt for question-answering
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

c:\Users\c1091\anaconda3\envs\llmrag\lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


## 4.8 Building RAG Workflow

The RAG we are building will primarily consist of two steps: Retrieval and Generation.

 - **Retrieval:** The question is used to perform a vector search, retrieving the most semantically relevant sub-documents from the previously created sub-documents.

 - **Generation:** The content of the retrieved sub-documents is provided to the LLM, which generates a complete answer based on the question.

In [28]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

from langgraph.graph import START, StateGraph

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

## 4.9 Done! Let's try it!

### 4.9.1 Some question you can try

```plaintext
Who was elected the 47th president of the United States?
```

```plaintext
What is the purpose of Trump's proposed tariffs on Taiwanese semiconductors?
```

```plaintext
What is DeepSeek, and why is it significant in the AI sector?
```

```plaintext
What impact has DeepSeek’s success had on big tech companies like Nvidia?
```

In [29]:
response = graph.invoke({"question": "Who was elected the 47th president of the United States?"})
print(response["answer"])

Donald Trump was elected the 47th president of the United States. He secured 71,289,216 votes nationwide, representing 51 percent of the total. This victory marked a historic comeback for the former president, who had previously been convicted of felony charges.


### 4.9.2 Trying harder question

This RAG we built is quite smart; you can try more questions in the `news-QA-dataset.json` file.

#### About 2024 Asian Baseball Championship Final

```plaintext
What was the score of Taiwan’s loss to South Korea in division B?
```

```plaintext
What action by South Korea’s pitcher allowed Su Yu-hsiang to steal second base in the final?
```

In [14]:
response = graph.invoke({"question": "What was the score of Taiwan’s loss to South Korea in division B?"})
print(response["answer"])

Taiwan lost to South Korea 0-1 in division B of the U-12 Asian Baseball Championship. However, Taiwan later defeated South Korea 5-1 in the final to win the championship. It was Taiwan's eighth championship in total.


# 5.RAG Use Case Overview

At this stage, we have successfully enabled the LLM to access the database we created and respond to queries based on the retrieved data.

However, for **public data** such as news searches, implementing RAG is not always necessary. Existing LLM products, such as ChatGPT, already integrate web search functionalities, allowing them to directly retrieve real-time public information.

### Key Insight: When to Use RAG
The **primary use case for RAG** lies in handling **enterprise data** or **offline data**. These types of **non-public data** are more suitable for RAG as a supplementary source for LLMs.

# 6.Enable Web Search Functionalities

In [35]:
#  pip install -U duckduckgo-search

## 6.1 Building Search-Enabled Agent  
Integrate a local LLM (Gemma3 via Ollama) with the DuckDuckGo Search tool to enable real-time web search functionalities.

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.chat_models import ChatOllama

# Create a local Gemma model
llm = ChatOllama(
    model="gemma3",
    temperature=1.0,
    max_tokens=1024,
    base_url="http://localhost:11434",
    timeout=30,
    max_retries=2,
)

# Original DuckDuckGo Tool
raw_search = DuckDuckGoSearchRun()

# Add a custom tool function for "no data record"
def safe_duckduckgo_search(query: str) -> str:
    result = raw_search.run(query)
    if not result.strip():
        return "查無結果，無法回答。"
    return result

# Setting tool list
tools = [
    Tool(
        name="DuckDuckGo Safe Search",
        func=safe_duckduckgo_search,
        description="用於即時搜尋問題答案，例如時事、股價、法律條文等"
    )
]

# Initialize Agent
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True,
)


## 6.2 Local Agent Now Performs Web Searches Before Answering

With the DuckDuckGo tool, our local LLM (Gemma3) now performs real-time web searches instead of fabricating answers.

This demonstrates the core benefit of RAG: to **significantly reduce hallucinations**.


In [39]:
response = agent.run("中華民國民法第1226條的內容是什麼？")
print(response)




> Entering new AgentExecutor chain...
I need to find the content of Article 1226 of the Civil Code of the People's Republic of China. I will use DuckDuckGo Safe Search to perform this task.
Action: DuckDuckGo Safe Search
Action Input: 中華民國民法第1226條

c:\Users\c1091\anaconda3\envs\llmrag\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(



Observation: 中華民國九十年五月九日 （本聲請書其餘附件略） 參考法條：中華民國憲法 第 22、23 條 (36.01.01) 民法 第 985、988、1050、1052、1056、1057 條 (91.06.26) 民事訴訟法 第 640 條 (89.02.09) 臺灣地區與大陸地區人民關係條例 第 64 條 (91.04.24) 資料來源： 民法 (民國 91 年 06 月 26 日) 非現行: 第 249 條: 定金，除當事人另有訂定外，適用左列之規定： 一 契約履行時，定金應返還或作為給付之一部。 二 契約因可歸責於付定金當事人之事由，致不能履行時，定金不得請求 返還。 三 契約因可歸責於受定金當事人之 ... 根據中華民國民法第365條的規定，買賣契約中關於瑕疵擔保請求權的時效有兩種： 短期時效： 買受人應於發現瑕疵後六個月內，向出賣人主張瑕疵擔保責任。此處的「發現」指的是買受人「實際知悉」瑕疵的存在，而非「可能或應知悉」。 二、本條配合第1003條之1之增訂 民法, 民法1026, 家庭生活費用, 生活費用, 負擔, 支付能力, 財產 所在編章︰ 第四編 親屬\第二章 婚姻\第四節 夫妻財產制\第二款 法定財產制（§1016~1030之4） 立法沿革︰ 19.12.26制定公布91.06.26修正公布 民法第1026條（刪除）（91.06.26 ... 听《《民法典》天天见!》上小宇宙。 与朋友们共同学习《民法典》，还有其他普法播客供选择，感兴趣可自行订阅。如需法律咨询可添加微信lawforalawyer，添加时请务必备注地区和案情，谢谢!
Thought:Thought: The provided text contains a large amount of information about the Civil Code of the People's Republic of China, including references to other articles and related laws. However, it doesn't explicitly state the content of Article 1226. It seems that the text is a summary and introductio

c:\Users\c1091\anaconda3\envs\llmrag\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(



Observation: 什麼是民律草案？ 《大清民律草案》是中國第一部近代民法典的草案。它於1910年完成，分為總則、債權、物權、親屬、繼承五編，共1596條。雖然因清朝覆亡而未及施行，但對中華民國《民法》的制定產生了深遠影響。 草案的歷史背景與意義 清末法制改革的產物：... 離婚財產分配是指在婚姻解除時，對夫妻共同財產進行合理分配的過程，依據《中華民國民法》第1058條規定，離婚時夫妻的共同財產應依照公平原則進行分配，法院考量雙方對家庭貢獻度及其他相關因素考量。 ... 深入分析113年度交簡字第1226號案件：詳細探討公共危險及其廣泛的法律後果。本頁面不僅提供案件的詳細審理過程，還深入解析相關法律條文，揭示這一判決對台灣司法制度的重大貢獻。這是法律專業人士、學者，以及對此類重要案件感興趣的公眾的理想資源，幫助您全面了解案件背景、法律脈絡 ... 听《《民法典》天天见!》上小宇宙。 与朋友们共同学习《民法典》，还有其他普法播客供选择，感兴趣可自行订阅。如需法律咨询可添加微信lawforalawyer，添加时请务必备注地区和案情，谢谢! 財政部臺北國稅局表示，依 遺產及贈與稅法第15條規定， 被繼承人死亡前2年內將財產贈與 配偶、民法第1138條及第1140條規定之各順序繼承人（即被繼承人之直系血親卑親屬、父母、兄弟姊妹及祖父母；直系血親卑親屬於繼承開始前死亡或喪失繼承權者，由其直系血親卑親屬代位繼承其應繼分）或該 ...
Thought:The provided text does not contain the content of Article 1226 of the Civil Code of the People's Republic of China. It primarily discusses the Qing Civil Code draft, divorce property distribution, and inheritance laws. I've exhausted searching with DuckDuckGo Safe Search.

Final Answer: I'm sorry, I cannot answer your question. The provided text does not contain th